In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, QuantoConfig, TrOCRProcessor, VisionEncoderDecoderModel
from transformers import pipeline

# ocr
pipe = pipeline("image-to-text", model="microsoft/trocr-large-handwritten")
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

# llm
model_id = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
quantization_config = QuantoConfig(weights="int8")
quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="mps:0", quantization_config=quantization_config)

In [9]:
from PIL import Image


def infer_text(image_path):
    image = Image.open(image_path).convert("RGB")
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    logits = model.generate(pixel_values, return_dict_in_generate=True, output_scores=True)
    return logits["scores"]
    # generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # return generated_text


infer_text('data/ss2.png')[0].shape

/Users/amaljoe/Desktop/Workspace/IITB/NLP/OCR_with_LLMs/.venv/lib/python3.9/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


torch.Size([1, 50265])

In [12]:
llm_input = "The quick brown fox jumps over the lazy dog"
llm_input_ids = tokenizer(llm_input, return_tensors="pt").input_ids
tokenizer.vocab_size

128000